In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from xgboost import XGBClassifier
from sklearn.svm import SVC

In [3]:
# Load dataset
data = pd.read_excel('Dataset.xlsx', engine='openpyxl')
data.head()

,Team,Match Up,Game Date,W/L,MIN,PTS,FGM,FGA,FG%,3PM,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-
0,GSW,GSW vs. PHX,10/24/2023,L,240,104,36,101,35.6,10,...,78.6,18,31,49,19,11,6,11,23,-4
1,PHX,PHX @ GSW,10/24/2023,W,240,108,42,95,44.2,11,...,76.5,17,43,60,23,5,7,19,22,4
2,LAL,LAL @ DEN,10/24/2023,L,240,107,41,90,45.6,10,...,75,13,31,44,23,5,4,12,18,-12
3,DEN,DEN vs. LAL,10/24/2023,W,240,119,48,91,52.7,14,...,75,9,33,42,29,9,6,12,15,12
4,MEM,MEM vs. NOP,10/25/2023,L,240,104,38,91,41.8,12,...,80,8,29,37,23,8,7,13,19,-7


Manipulate Data:

In [4]:
# Convert game date to datetime
data['Game Date'] = pd.to_datetime(data['Game Date'], format='%m/%d/%Y')

# Extract home and guest teams
data['Home Team'] = data['Match Up'].apply(lambda x: x.split(' vs. ')[0] if 'vs.' in x else x.split(' @ ')[1])
data['Guest Team'] = data['Match Up'].apply(lambda x: x.split(' vs. ')[1] if 'vs.' in x else x.split(' @ ')[0])

# Add the correct Label column: 1 if home team won, 0 otherwise
data['Label'] = data.apply(
    lambda row: 1 if ((row['Team'] == row['Home Team']) and (row['W/L'] == 'W')) 
                    or ((row['Team'] == row['Guest Team']) and (row['W/L'] == 'L'))
                else 0,
    axis=1
)
data.head()

,Team,Match Up,Game Date,W/L,MIN,PTS,FGM,FGA,FG%,3PM,...,REB,AST,STL,BLK,TOV,PF,+/-,Home Team,Guest Team,Label
0,GSW,GSW vs. PHX,2023-10-24,L,240,104,36,101,35.6,10,...,49,19,11,6,11,23,-4,GSW,PHX,0
1,PHX,PHX @ GSW,2023-10-24,W,240,108,42,95,44.2,11,...,60,23,5,7,19,22,4,GSW,PHX,0
2,LAL,LAL @ DEN,2023-10-24,L,240,107,41,90,45.6,10,...,44,23,5,4,12,18,-12,DEN,LAL,1
3,DEN,DEN vs. LAL,2023-10-24,W,240,119,48,91,52.7,14,...,42,29,9,6,12,15,12,DEN,LAL,1
4,MEM,MEM vs. NOP,2023-10-25,L,240,104,38,91,41.8,12,...,37,23,8,7,13,19,-7,MEM,NOP,0


Calculating bias for later adding Home Advatange feature:

But don't use in the end because we've already have half of the data to make home advantage's information is in the model.

In [5]:
# Create a binary feature for home games
data['Is_Home'] = data['Match Up'].str.contains(' vs. ').astype(int)

# Prepare features and labels
X = data[['Is_Home']]  # Home/Away indicator
y = (data['W/L'] == 'W').astype(int)  # Convert W/L to binary outcome

# Fit a linear regression model
model = LinearRegression()
model.fit(X, y)

# The coefficient of 'Is_Home' represents the home advantage bias
bias = model.coef_[0]
print(f"Calculated Bias (Regression): {bias}")
new_bias = 1/21 * bias

# Remove the 'Is_Home' column
data = data.drop(columns=['Is_Home'])

Calculated Bias (Regression): 0.08617886178861786


This df below only contains stability features. Here is how this feature were computed:

stability score of the feature = mean performance of the feature / variance of the feature

stability score of the team = average of the stability score for each features

difference in average stability score (what we finally used) = stability score of home team - stability score of guest team

In [6]:
# dataset with new feature: stability score
# Initialize the new dataset
new_dataset = []

# Iterate over each match-up
for _, row in data.iterrows():
    game_date = row['Game Date']
    home_team = row['Home Team']
    guest_team = row['Guest Team']
    
    # Filter games before the current game date for both teams
    home_team_games = data[(data['Team'] == home_team) & (data['Game Date'] < game_date)]
    guest_team_games = data[(data['Team'] == guest_team) & (data['Game Date'] < game_date)]
    
    # Calculate W/L rates (win rates) for both teams
    home_team_wins = (home_team_games['W/L'] == 'W').sum()
    home_team_total = len(home_team_games)
    guest_team_wins = (guest_team_games['W/L'] == 'W').sum()
    guest_team_total = len(guest_team_games)
    
    home_win_rate = home_team_wins / home_team_total if home_team_total > 0 else 0
    guest_win_rate = guest_team_wins / guest_team_total if guest_team_total > 0 else 0
    
    # Calculate W/L difference
    wl_difference = home_win_rate - guest_win_rate
    
    # Calculate Stability using mean and variance
    if not home_team_games.empty:
        home_mean_performance = home_team_games.iloc[:, 5:].mean(numeric_only=True)
        home_variance_performance = home_team_games.iloc[:, 5:].var(numeric_only=True)
        home_stability = (home_mean_performance / (home_variance_performance + 1e-6)).mean()
        home_stability = min(max(home_stability, -100), 100)
    else:
        home_stability = 0

    if not guest_team_games.empty:
        guest_mean_performance = guest_team_games.iloc[:, 5:].mean(numeric_only=True)
        guest_variance_performance = guest_team_games.iloc[:, 5:].var(numeric_only=True)
        guest_stability = (guest_mean_performance / (guest_variance_performance + 1e-6)).mean()
        guest_stability = min(max(guest_stability, -100), 100)  # Limit stability within [-100, 100]
    else:
        guest_stability = 0

    # Stability difference
    stability = home_stability - guest_stability
    
    # Prepare the new row
    new_row = {
        'Game Date': game_date,
        'Home Team': home_team,
        'Guest Team': guest_team,
        'Label': int(row['Label']),  # Ensure Label is directly copied and kept as integer
        'W/L Difference': wl_difference,
        'Stability': stability
    }
    
    # Add statistics differences (like in the original approach)
    home_team_stats = home_team_games.iloc[:, 4:].mean(numeric_only=True)
    guest_team_stats = guest_team_games.iloc[:, 4:].mean(numeric_only=True)
    
    if not home_team_stats.empty and not guest_team_stats.empty:
        stats_diff = home_team_stats - guest_team_stats
        # Update new_row but ensure no conflict with 'Label'
        new_row.update({k: v for k, v in stats_diff.to_dict().items() if k != 'Label'})
    
    new_dataset.append(new_row)
# Convert to DataFrame
new_dataset_df_1 = pd.DataFrame(new_dataset)

# Save to file for inspection
output_file = 'output_dataset/Dataset_With_Stability.xlsx'
new_dataset_df_1.to_excel(output_file, index=False)

output_file

'output_dataset/Dataset_With_Stability.xlsx'

This fucntion and generated df below contain stability feature, Previous Competitions and Home-Away Win Rate Difference.

Home-Away Win Rate Difference: I took the home team's past home win percentage minus the away team's past away win percentage;

Previous Competitions: Just like the discreption in guideline.

In [7]:
def standardize_match(row):
    teams = sorted([row['Home Team'], row['Guest Team']])
    standardized_row = {
        'Standard Home Team': teams[0],
        'Standard Guest Team': teams[1],
        'Game Date': row['Game Date'],
        'W/L': 'W' if (row['Home Team'] == teams[0] and row['W/L'] == 'W') or
                       (row['Guest Team'] == teams[0] and row['W/L'] == 'L') else 'L'
    }
    return standardized_row

In [54]:
# Initialize the new dataset
new_dataset = []

# Iterate over each match-up
for _, row in data.iterrows():
    game_date = row['Game Date']
    home_team = row['Home Team']
    guest_team = row['Guest Team']
    
    # Filter games before the current game date for both teams
    home_team_games = data[(data['Team'] == home_team) & (data['Game Date'] < game_date)]
    guest_team_games = data[(data['Team'] == guest_team) & (data['Game Date'] < game_date)]

    # Calculate home team's historical home win rate
    home_team_home_games = home_team_games[home_team_games['Match Up'].str.contains('vs.')]
    home_team_home_wins = (home_team_home_games['W/L'] == 'W').sum()
    home_team_home_total = len(home_team_home_games)
    home_team_home_win_rate = home_team_home_wins / home_team_home_total if home_team_home_total > 0 else 0

    # Calculate guest team's historical away win rate
    guest_team_away_games = guest_team_games[guest_team_games['Match Up'].str.contains('@')]
    guest_team_away_wins = (guest_team_away_games['W/L'] == 'W').sum()
    guest_team_away_total = len(guest_team_away_games)
    guest_team_away_win_rate = guest_team_away_wins / guest_team_away_total if guest_team_away_total > 0 else 0

    home_away_win_rate_diff = home_team_home_win_rate - guest_team_away_win_rate

    # Filter previous competitions between home and guest teams
    previous_matches = data[
        ((data['Home Team'] == home_team) & (data['Guest Team'] == guest_team)) |
        ((data['Home Team'] == guest_team) & (data['Guest Team'] == home_team))
    ]
    previous_matches = previous_matches[previous_matches['Game Date'] < game_date]

    standardized_matches = previous_matches.apply(standardize_match, axis=1, result_type='expand')

    standardized_matches = standardized_matches.drop_duplicates(subset=['Game Date', 'Standard Home Team', 'Standard Guest Team'])

    # Calculate Previous Competitions feature (only direct matches between home and guest teams)
    previous_competitions_score = 0
    for _, match in standardized_matches.iterrows():
        if match['Standard Home Team'] == home_team and match['W/L'] == 'W':
            previous_competitions_score += 10
        elif match['Standard Home Team'] == home_team and match['W/L'] == 'L':
            previous_competitions_score -= 10
    
    # Calculate W/L rates (win rates) for both teams
    home_team_wins = (home_team_games['W/L'] == 'W').sum()
    home_team_total = len(home_team_games)
    guest_team_wins = (guest_team_games['W/L'] == 'W').sum()
    guest_team_total = len(guest_team_games)
    
    home_win_rate = home_team_wins / home_team_total if home_team_total > 0 else 0
    guest_win_rate = guest_team_wins / guest_team_total if guest_team_total > 0 else 0
    
    # Calculate W/L difference
    wl_difference = home_win_rate - guest_win_rate
    
    # Calculate Stability using mean and variance
    if not home_team_games.empty:
        home_mean_performance = home_team_games.iloc[:, 4:].mean(numeric_only=True)
        home_variance_performance = home_team_games.iloc[:, 4:].var(numeric_only=True)
        home_stability = (home_mean_performance / (home_variance_performance + 1e-6)).mean()
        home_stability = min(max(home_stability, -100), 100)
    else:
        home_stability = 0

    if not guest_team_games.empty:
        guest_mean_performance = guest_team_games.iloc[:, 4:].mean(numeric_only=True)
        guest_variance_performance = guest_team_games.iloc[:, 4:].var(numeric_only=True)
        guest_stability = (guest_mean_performance / (guest_variance_performance + 1e-6)).mean()
        guest_stability = min(max(guest_stability, -100), 100)  # Limit stability within [-100, 100]
    else:
        guest_stability = 0

    # Stability difference
    stability = home_stability - guest_stability
    
    # Prepare the new row
    new_row = {
        'Game Date': game_date,
        'Home Team': home_team,
        'Guest Team': guest_team,
        'Label': int(row['Label']),  # Ensure Label is directly copied and kept as integer
        'W/L Difference': wl_difference,
        'Stability': stability,
        'Previous Competitions': previous_competitions_score,  # Add new feature
        'Home-Away Win Rate Difference': home_away_win_rate_diff
    }
    
    # Add statistics differences (like in the original approach)
    home_team_stats = home_team_games.iloc[:, 4:].mean(numeric_only=True)
    guest_team_stats = guest_team_games.iloc[:, 4:].mean(numeric_only=True)
    
    if not home_team_stats.empty and not guest_team_stats.empty:
        stats_diff = home_team_stats - guest_team_stats
        # Update new_row but ensure no conflict with 'Label'
        new_row.update({k: v for k, v in stats_diff.to_dict().items() if k != 'Label'})
    
    new_dataset.append(new_row)

# Convert to DataFrame
new_dataset_df_1 = pd.DataFrame(new_dataset)

# Save to file for inspection
output_file = 'output_dataset/Dataset_with_Stability_and_Previous_Competitions_Corrected.xlsx'
new_dataset_df_1.to_excel(output_file, index=False)

output_file

'output_dataset/Dataset_with_Stability_and_Previous_Competitions_Corrected.xlsx'

This code below doesn't work. I try to add some weights when I calculating the average statistics, but it failed... You may try to revise it?

In [29]:
import numpy as np

# Define a weighting function (e.g., exponential decay)
def calculate_weight(days_since_match, alpha=0.1):
    """Calculate weight based on days since match."""
    return np.exp(-alpha * days_since_match)+1

In [59]:
print(new_dataset_df_1['Stability'].dropna())
dataset_df_1 = new_dataset_df_1.drop_duplicates().dropna()
columns1 = new_dataset_df_1['Stability'].dropna()
columns2 = new_dataset_df_1['Previous Competitions'].dropna()

0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       0.000000
          ...   
2455   -0.503363
2456    1.437285
2457    1.352189
2458    1.279545
2459   -0.221080
Name: Stability, Length: 2424, dtype: float64


In [53]:
print(columns2)

0       0
1       0
2       0
3       0
4       0
       ..
2455    0
2456    0
2457    0
2458    0
2459    0
Name: Previous Competitions, Length: 2460, dtype: int64


In [45]:
# Initialize the new dataset
new_dataset = []

# Iterate over each match-up
for _, row in data.iterrows():
    game_date = row['Game Date']
    home_team = row['Home Team']
    guest_team = row['Guest Team']
    
    # Directly preserve the original label (0 or 1)
    label = int(row['Label'])

    # Filter games before the current game date for both teams
    home_team_games = data[(data['Team'] == home_team) & (data['Game Date'] < game_date)].copy()
    guest_team_games = data[(data['Team'] == guest_team) & (data['Game Date'] < game_date)].copy()
    
    # Calculate days since each previous match
    home_team_games['Days Since Match'] = (game_date - home_team_games['Game Date']).dt.days
    guest_team_games['Days Since Match'] = (game_date - guest_team_games['Game Date']).dt.days

    # Apply weighting function
    home_team_games['Weight'] = calculate_weight(home_team_games['Days Since Match'])
    guest_team_games['Weight'] = calculate_weight(guest_team_games['Days Since Match'])

    # Filter numeric columns for weighted calculations
    home_numeric_cols = home_team_games.select_dtypes(include=['number']).copy()
    guest_numeric_cols = guest_team_games.select_dtypes(include=['number']).copy()

    # Ensure weight column exists and is numeric
    home_team_games['Weight'] = pd.to_numeric(home_team_games['Weight'], errors='coerce').fillna(0)
    guest_team_games['Weight'] = pd.to_numeric(guest_team_games['Weight'], errors='coerce').fillna(0)

    # Weighted average calculation for home team
    if not home_numeric_cols.empty and not home_team_games.empty:
        home_team_stats = (
            home_numeric_cols.multiply(home_team_games['Weight'], axis=0).sum()
            / home_team_games['Weight'].sum()
        )
    else:
        home_team_stats = pd.Series(0, index=home_numeric_cols.columns)

    # Weighted average calculation for guest team
    if not guest_numeric_cols.empty and not guest_team_games.empty:
        guest_team_stats = (
            guest_numeric_cols.multiply(guest_team_games['Weight'], axis=0).sum()
            / guest_team_games['Weight'].sum()
        )
    else:
        guest_team_stats = pd.Series(0, index=guest_numeric_cols.columns)

    # Calculate weighted W/L rates
    home_team_wins = (home_team_games['W/L'] == 'W').astype(float)
    home_weighted_win_rate = (home_team_wins * home_team_games['Weight']).sum() / home_team_games['Weight'].sum() if not home_team_games.empty else 0

    guest_team_wins = (guest_team_games['W/L'] == 'W').astype(float)
    guest_weighted_win_rate = (guest_team_wins * guest_team_games['Weight']).sum() / guest_team_games['Weight'].sum() if not guest_team_games.empty else 0

    wl_difference = home_weighted_win_rate - guest_weighted_win_rate

    # Calculate Stability using weighted mean and variance
    if not home_team_games.empty:
        home_stability = (
            home_team_games['Weight'] * home_team_games.iloc[:, 4:].var(numeric_only=True)
        ).mean()
    else:
        home_stability = 0

    if not guest_team_games.empty:
        guest_stability = (
            guest_team_games['Weight'] * guest_team_games.iloc[:, 4:].var(numeric_only=True)
        ).mean()
    else:
        guest_stability = 0

    stability = home_stability - guest_stability

    # Calculate Previous Competitions feature (as before)
    previous_matches = data[
        ((data['Home Team'] == home_team) & (data['Guest Team'] == guest_team)) |
        ((data['Home Team'] == guest_team) & (data['Guest Team'] == home_team))
    ]
    previous_matches = previous_matches[previous_matches['Game Date'] < game_date]
    previous_competitions_score = 0
    for _, match in previous_matches.iterrows():
        if match['Home Team'] == home_team and match['W/L'] == 'W':
            previous_competitions_score += 10
        elif match['Home Team'] == home_team and match['W/L'] == 'L':
            previous_competitions_score -= 10
    
    print(previous_competitions_score)

    # Prepare the new row
    new_row = {
        'Game Date': game_date,
        'Home Team': home_team,
        'Guest Team': guest_team,
        'Label': label,  # Use the original label directly
        'W/L Difference': wl_difference,
        'Stability': stability,
        'Previous Competitions': int(previous_competitions_score),  # Ensure Previous Competitions remains integer
    }

    # Add weighted stats differences to new row
    stats_diff = home_team_stats - guest_team_stats
    if not stats_diff.empty:
    # Ensure we don't overwrite 'Label' and 'Previous Competitions'
        stats_diff_filtered = {k: v for k, v in stats_diff.to_dict().items() if k not in ['Label', 'Previous Competitions']}
        new_row.update(stats_diff_filtered)

    # Append to the dataset
    new_dataset.append(new_row)

# Convert to DataFrame
new_dataset_df_1 = pd.DataFrame(new_dataset)

# Save to file for inspection
output_file = 'output_dataset/Dataset_with_Weights.xlsx'
new_dataset_df_1.to_excel(output_file, index=False)

output_file

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


'output_dataset/Dataset_with_Weights.xlsx'

Always run the below chunck to make sure get a clean dataset (without Na and duplicates).

In [52]:
# Remove rows with NaN values and drop duplicate rows
# dataset_df_1 = new_dataset_df_1.dropna().drop_duplicates()
dataset_df_1 = new_dataset_df_1
dataset_df_1 = dataset_df_1.iloc[29:]
dataset_df_1 = dataset_df_1.drop(index=30).reset_index(drop=True)
dataset_df_1 = dataset_df_1.drop_duplicates()
dataset_df_1['Stability'] = columns1
dataset_df_1['Previous Competitions'] = columns2
dataset_df_1.head()

,Game Date,Home Team,Guest Team,Label,W/L Difference,Stability,Previous Competitions,MIN,PTS,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,+/-,Days Since Match,Weight
0,2023-10-26,LAL,PHX,1,-1.0,0.0,0,0.0,-1.0,-1.0,...,-12.0,-16.0,0.0,0.0,-3.0,-7.0,-4.0,-16.0,0.0,0.000000
2,2023-10-27,SAS,HOU,1,0.0,0.0,0,0.0,33.0,14.0,...,12.0,14.0,14.0,2.0,2.0,0.0,-1.0,23.0,0.0,0.000000
3,2023-10-27,BOS,MIA,1,0.0,0.0,0,0.0,5.0,0.0,...,7.0,-2.0,-4.0,-5.0,8.0,6.0,4.0,3.0,0.0,0.000000
4,2023-10-27,UTA,LAC,1,-1.0,0.0,0,0.0,-9.0,-6.0,...,7.0,9.0,-14.0,-5.0,-5.0,-4.0,0.0,-28.0,0.0,0.000000
5,2023-10-27,SAC,GSW,0,1.0,0.0,0,0.0,26.0,11.0,...,-1.0,-4.0,10.0,-5.0,2.0,1.0,2.0,20.0,-1.0,0.077913


In [46]:
# Remove rows with NaN values and drop duplicate rows
# dataset_df_1 = new_dataset_df_1.dropna().drop_duplicates()
dataset_df_1 = new_dataset_df_1
dataset_df_1 = dataset_df_1.iloc[29:]
dataset_df_1 = dataset_df_1.drop(index=1).reset_index(drop=True)
dataset_df_1 = dataset_df_1.drop_duplicates()
dataset_df_1['Stability'] = columns1
dataset_df_1['Previous Competitions'] = columns2
dataset_df_1.head()
from sklearn.preprocessing import MinMaxScaler

# Step 1: Select columns starting from the 5th column onward
features_to_normalize = dataset_df_1.iloc[:, 4:]  # Select all columns from the 5th column onward (0-indexed)

# Step 2: Apply Min-Max normalization
scaler = MinMaxScaler()
normalized_features = scaler.fit_transform(features_to_normalize)

# Step 3: Replace the original columns with the normalized ones
dataset_df_1.iloc[:, 4:] = normalized_features

# Save to file for inspection
output_file = 'output_dataset/cleaned_final_dataset_with_features.xlsx'
dataset_df_1.to_excel(output_file, index=False)
output_file

KeyError: '[1] not found in axis'

In [40]:
X = dataset_df_1.drop(columns=['Label', 'Game Date', 'Home Team', 'Guest Team', 'Stability']) # Features: all columns except 'Label'
y = dataset_df_1['Label']  # Labels: the 'Label' column
X = X[100:]
y = y[100:] # remove first 100 rows to make sure the stability are a lot 0.
X.head()

,W/L Difference,Previous Competitions,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,...,DREB,REB,AST,STL,BLK,TOV,PF,+/-,Days Since Match,Weight
595,0.597461,0,0.084775,0.589604,0.569851,0.547862,0.590117,0.673573,0.657608,0.527251,...,0.417901,0.180372,0.446557,0.393120,0.404486,0.000000,0.403924,0.622708,0.486829,0.498149
597,0.403925,0,0.746078,0.405397,0.550640,0.826151,0.408239,0.387970,0.531841,0.057293,...,0.625031,0.774333,0.650523,0.474192,0.729047,0.660301,0.481401,0.441260,0.462278,0.544677
598,0.348062,0,0.637099,0.425914,0.328232,0.455321,0.324600,0.509742,0.522707,0.412075,...,0.548636,0.493831,0.221828,0.650519,0.492034,0.598977,0.678122,0.482406,0.422663,0.606381
599,0.326012,0,0.403362,0.535457,0.607204,0.482199,0.695127,0.329561,0.181722,0.595868,...,0.582288,0.608803,0.680070,0.280205,0.397625,0.785329,0.668880,0.403685,0.513353,0.489160
604,0.476502,0,0.753511,0.343028,0.355746,0.455593,0.368094,0.514517,0.602455,0.293503,...,0.398030,0.321524,0.385487,0.631454,0.529323,0.441442,0.495219,0.562413,0.506457,0.512665


Using data standardization and L1 regularization to complete Feature Selection.

In [41]:
# Step 1: Standardize the features
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)  # Standardize X to have zero mean and unit variance

# Step 2: Fit Logistic Regression with L1 regularization for binary model
lasso_log_reg = LogisticRegression(penalty='l1', solver='liblinear', C=1.0, random_state=42)  # L1 penalty for feature selection
lasso_log_reg.fit(X_standardized, y)

# Step 3: Identify selected features
selected_features = X.columns[(lasso_log_reg.coef_ != 0).flatten()]  # Keep only features with non-zero coefficients
X_selected = X[selected_features]  # Subset original X with selected features

print(f"Selected features: {list(selected_features)}")
#X_selected = X[['Stability', 'Home-Away Win Rate Difference', 'FG%', '3P%', '+/-']]
X_selected.head()

Selected features: ['W/L Difference', 'MIN', 'FGA', 'FG%', '3PA', '3P%', 'FTM', 'FTA', 'OREB', 'DREB', 'AST', 'STL', 'TOV', '+/-', 'Days Since Match', 'Weight']


,W/L Difference,MIN,FGA,FG%,3PA,3P%,FTM,FTA,OREB,DREB,AST,STL,TOV,+/-,Days Since Match,Weight
595,0.597461,0.084775,0.547862,0.590117,0.657608,0.527251,0.565053,0.685705,0.271741,0.417901,0.446557,0.393120,0.000000,0.622708,0.486829,0.498149
597,0.403925,0.746078,0.826151,0.408239,0.531841,0.057293,0.304923,0.438364,0.876222,0.625031,0.650523,0.474192,0.660301,0.441260,0.462278,0.544677
598,0.348062,0.637099,0.455321,0.324600,0.522707,0.412075,0.775910,0.714332,0.566643,0.548636,0.221828,0.650519,0.598977,0.482406,0.422663,0.606381
599,0.326012,0.403362,0.482199,0.695127,0.181722,0.595868,0.594880,0.588698,0.690902,0.582288,0.680070,0.280205,0.785329,0.403685,0.513353,0.489160
604,0.476502,0.753511,0.455593,0.368094,0.602455,0.293503,0.471495,0.447432,0.493143,0.398030,0.385487,0.631454,0.441442,0.562413,0.506457,0.512665


Training & Testing using 5-fold cv (including Random Forest, Logistic Regression, Decision Tree, AdaBoost, and QDA):

The accuracy of logistic regression is the highest which is about 0.66 for either 1 new feature or 3 new features. So we imporved 0.2 in total...

In [42]:
# Ensure data is shuffled before splitting
X_shuffled, y_shuffled = shuffle(X_selected, y, random_state=0)

# Initialize models
models = {
    'Random Forest': RandomForestClassifier(random_state=0),
    'Logistic Regression': LogisticRegression(random_state=0, max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(random_state=0),
    'AdaBoost': AdaBoostClassifier(random_state=0),
    #'QDA': QuadraticDiscriminantAnalysis(),
    'XGBoost': XGBClassifier(eval_metric='logloss', random_state=0),
    'Kernel SVM': SVC(kernel='rbf', random_state=0),
    'Linear SVM': SVC(kernel='linear', random_state=0)  # Add kernel SVM
}

# Initialize K-Fold with shuffle
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# Store accuracies for each model
model_accuracies = {model_name: [] for model_name in models}

for train_index, test_index in kf.split(X_shuffled):
    # Split the data into training and testing sets
    X_train, X_test = X_shuffled.iloc[train_index], X_shuffled.iloc[test_index]
    y_train, y_test = y_shuffled.iloc[train_index], y_shuffled.iloc[test_index]
    
    for model_name, model in models.items():
        # Fit the model
        model.fit(X_train, y_train)
        
        # Predict on the test set
        y_pred = model.predict(X_test)
        
        # Calculate accuracy
        accuracy = accuracy_score(y_test, y_pred)
        model_accuracies[model_name].append(accuracy)

# Print results
for model_name, accuracies in model_accuracies.items():
    print(f"{model_name} Accuracy for each fold: {accuracies}")
    print(f"{model_name} Mean accuracy: {sum(accuracies) / len(accuracies):.2f}")

Random Forest Accuracy for each fold: [0.6075268817204301, 0.6720430107526881, 0.6182795698924731, 0.6182795698924731, 0.6236559139784946]
Random Forest Mean accuracy: 0.63
Logistic Regression Accuracy for each fold: [0.6451612903225806, 0.7043010752688172, 0.6451612903225806, 0.6881720430107527, 0.5806451612903226]
Logistic Regression Mean accuracy: 0.65
Decision Tree Accuracy for each fold: [0.553763440860215, 0.6451612903225806, 0.5967741935483871, 0.5376344086021505, 0.532258064516129]
Decision Tree Mean accuracy: 0.57
AdaBoost Accuracy for each fold: [0.6505376344086021, 0.6075268817204301, 0.5860215053763441, 0.5698924731182796, 0.6290322580645161]
AdaBoost Mean accuracy: 0.61
XGBoost Accuracy for each fold: [0.5483870967741935, 0.6827956989247311, 0.6182795698924731, 0.6129032258064516, 0.6021505376344086]
XGBoost Mean accuracy: 0.61
Kernel SVM Accuracy for each fold: [0.6344086021505376, 0.6881720430107527, 0.6397849462365591, 0.6451612903225806, 0.6129032258064516]
Kernel SVM 

The code below is about a neural network without using k-fold.

The accuracy of data with only stability is about 0.66.

The accuracy of data with stability, previous competitionm and Home-Away Win Rate Difference is also about 0.66.

In [35]:
# Step 1: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=8)

X_train = X_train[100:]
y_train = y_train[100:]

# Step 2: Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 3: Build the neural network model
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),  # First hidden layer
    Dropout(0.3),  # Dropout for regularization
    Dense(32, activation='relu'),  # Second hidden layer
    Dense(1, activation='sigmoid')  # Output layer (binary classification)
])

# Step 4: Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 5: Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Step 6: Evaluate the model on the test set
y_pred = (model.predict(X_test) > 0.5).astype(int)  # Convert probabilities to binary predictions
accuracy = accuracy_score(y_test, y_pred)

print(f"Neural Network Test Accuracy: {accuracy:.2f}")


Epoch 1/20
21/21 [==============================] - 0s 7ms/step - loss: 0.6696 - accuracy: 0.5712 - val_loss: 0.6619 - val_accuracy: 0.6030
Epoch 2/20
21/21 [==============================] - 0s 4ms/step - loss: 0.6244 - accuracy: 0.6537 - val_loss: 0.6450 - val_accuracy: 0.6303
Epoch 3/20
21/21 [==============================] - 0s 5ms/step - loss: 0.6196 - accuracy: 0.6552 - val_loss: 0.6429 - val_accuracy: 0.6273
Epoch 4/20
21/21 [==============================] - 0s 5ms/step - loss: 0.6026 - accuracy: 0.6732 - val_loss: 0.6393 - val_accuracy: 0.6303
Epoch 5/20
21/21 [==============================] - 0s 4ms/step - loss: 0.6063 - accuracy: 0.6537 - val_loss: 0.6374 - val_accuracy: 0.6394
Epoch 6/20
21/21 [==============================] - 0s 5ms/step - loss: 0.6085 - accuracy: 0.6657 - val_loss: 0.6379 - val_accuracy: 0.6333
Epoch 7/20
21/21 [==============================] - 0s 4ms/step - loss: 0.5942 - accuracy: 0.6777 - val_loss: 0.6360 - val_accuracy: 0.6333
Epoch 8/20
21/21 [==